# Cancer Relapse Classification Using Gene Expression Profiles

One really interesting problem in computational molecular medicine, and one that is heavily researched is the prediction of phenotypes (e.g. properties of cancer growths) based on gene expression profiles.

For this project, I try to predict the time of relapse among breast cancer patients diagnosed with a malignant tumor. In particular, I consider patients with ER+ (estrogen receptor positive) disease who were treated with surgery or surgery and radiation, and untreated with systemic hormonal therapy and/or chemotherapy. Of these, I consider 2 categories of patients: 'NoRelapse' patients who did not relapse for the duration of the 6.5 year study and 'Relapse' patients who did. The researchers  did post-study follow-ups, and there were indeed patients in the study who relapsed after 6.5 years, but they have been excluded from my data.

The data come from DNA microarrays. The data compose a matrix where each row represents an anonymized patient and each column represents the expression level for a particular gene.

Without diving too deep into the biochemical methods behind gathering the data, I just wanted to provide a brief overview of what 'DNA microarray data' actually is. A microarray is a collection of small DNA spots attached to a solid surface. In microarray experiments, the signal collected from each spot is used to estimate the expression level of a gene. A microarray contains thousands of DNA spots, covering almost every gene in a genome.

To make the gene expression levels more comparable between genes, the data have been normalized. This makes it so the numbers in the dataframe are not the *true* expression levels, but are slighly altered so we can interpret and compare these numbers across genes (the I gathered the normalized data, so I did not have to do any extra preprocessing beforehand).

I was particularly lucky because the data I gathered was already clean. There were no missing variables, and as I mentioned before, the were already normalized for interpretability.

I then split the data into 2 categories: a training set (the set of patients I use to build my predictive model) and a testing set (the set of patients I use to evaluate how well my model performs). The training set contain 22,215 genes for 212 patients. Of these, 152 patients are categorized as 'NoRelapse' and 60 are categorized as 'Relapse'. The test set also contain 22,215 genes for 212 patients, and of these, 137 are categorized as 'NoRelapse' and 75 as 'Relapse'.

As with any data science problem, I begin by importing the Python tools and libraries I need:

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import math
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline
from sklearn import cross_validation
from time import time
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, classification_report

Below, you can see the output of what the gene expression data look like. On the top left, you see 'GENE' which indicates that each column represents a gene. For example, the first gene in the matrix is '1007_s_at'. This is not acutally a gene, but rather a specific gene sequence. The second column is labeled '10543_at;RFC2'. RFC2 is the actual name of the gene, and that gene was interrogated by probe 1053_at. Since not every gene is labeled (like the first column), I decided to keep both the probe and gene name to identify genes.

Each row is labeled with 'GSM 177###". This code represents a patient. Thus, you see the first patient 'GSM 177886' has 12.2321 expressed for '1007_s_at', 10.4610 for gene RFC2, etc.

For a high-level description of what probes are in this context, I turned to the Internet:
"Every probe is in essence a gene, since every probe interrogates an expressed sequence. Some probes may not have any annotation associated with them, some probes may recognize multiple potential target sequences (promiscuous probes) and some annotated genes may have multiple probes, each of with interrogates a different mRNA transcript for that gene." - Michael B. Black https://www.researchgate.net/post/Which_probe_set_should_I_consider_for_each_gene_in_affymetrix_microarray

In [2]:
df = pd.read_table('../data/brcaTrainExpr.txt', index_col = 'GENE')
df = df.transpose()
X = np.array(df)
print(df.shape)
df.head()

(212, 22215)


GENE,1007_s_at,1053_at;RFC2,117_at;HSPA6,121_at;PAX8,1255_g_at;GUCA1A,1294_at,1316_at;THRA,1320_at;PTPN21,1405_i_at;CCL5,1431_at;CYP2E1,...,90610_at;LRCH4,91580_at;LRTM1,91617_at,91682_at;EXOSC4,91684_g_at;EXOSC4,91703_at;EHBP1L1,91816_f_at;MEX3D,91826_at;EPS8L1,91920_at;BCAN,91952_at;DCAF15
GSM177886,12.2321,10.4610,9.1020,10.1887,6.4191,9.8911,6.6967,7.2187,10.9496,5.1459,...,10.3830,6.3388,8.8040,8.8703,10.6404,8.6402,8.6949,9.4653,8.5877,8.9001
GSM177888,12.7247,10.2661,8.2479,10.7472,6.8486,10.2362,8.9413,7.6068,12.8489,6.4327,...,11.0849,4.0472,8.7750,8.6050,11.6073,9.3611,8.4957,9.6736,9.2171,7.6720
GSM177890,12.0137,9.9722,7.4331,10.3992,4.4546,10.1440,7.9000,7.6847,11.4060,6.2453,...,10.1170,4.6438,8.2863,8.5939,10.2225,8.8233,8.6582,9.5091,9.2464,8.4098
GSM177893,12.4415,10.2573,8.2079,10.9866,3.7594,10.2328,8.3606,5.7863,7.4300,7.8563,...,10.3650,3.3177,8.6374,8.7150,10.1531,7.3304,8.5776,9.5751,9.1949,8.2414
GSM177894,12.3090,10.1558,6.0089,10.2796,6.8940,9.7229,7.7062,4.9384,10.5943,6.7776,...,10.3119,5.6703,8.7074,8.5682,9.6210,8.6251,9.1388,10.2063,8.5504,8.3415


In [3]:
dfT = pd.read_table('../data/brcaTestExpr.txt', index_col = 'GENE')
dfT = dfT.transpose()
XT = np.array(dfT)
print(dfT.shape)
dfT.head()

(212, 22215)


GENE,1007_s_at,1053_at;RFC2,117_at;HSPA6,121_at;PAX8,1255_g_at;GUCA1A,1294_at,1316_at;THRA,1320_at;PTPN21,1405_i_at;CCL5,1431_at;CYP2E1,...,90610_at;LRCH4,91580_at;LRTM1,91617_at,91682_at;EXOSC4,91684_g_at;EXOSC4,91703_at;EHBP1L1,91816_f_at;MEX3D,91826_at;EPS8L1,91920_at;BCAN,91952_at;DCAF15
GSM177920,12.3029,9.6711,8.2329,9.8180,6.5842,9.7000,7.6245,5.3078,8.6243,7.8594,...,9.8653,4.6365,9.2609,8.0202,9.6465,7.7720,9.3198,11.7674,9.5955,8.5581
GSM177969,11.8008,10.5399,8.2761,9.9577,5.5922,10.1655,7.7667,7.2959,10.6950,7.5074,...,9.8680,7.0481,8.8045,8.8985,10.0809,8.3769,9.5190,9.8512,9.1547,8.2553
GSM177983,12.5286,9.0396,8.7496,10.2907,6.9988,10.4279,7.7974,5.4544,8.2786,7.2004,...,9.0900,6.6631,8.9407,8.6171,9.9880,8.7302,8.4114,9.5867,9.5263,8.6281
GSM177988,10.5332,9.8496,8.0805,9.8766,5.6708,11.8509,8.7876,7.7491,10.4160,7.8056,...,11.5998,2.9187,8.8900,8.2380,8.8388,9.8467,6.8486,6.3113,8.3880,9.2349
GSM178002,12.3055,9.8148,9.4102,10.3765,7.3082,10.5140,7.9715,7.8410,10.8660,6.7951,...,10.4178,5.9855,8.9733,8.7199,9.8251,9.2594,9.1781,9.7976,9.1203,8.0822


Below, I import the label, which is whether or not the patient relapsed. I redefine Relapse as 1 and NoRelapse as 0.

In [4]:
dfy = pd.read_table('../data/brcaTrainPheno.txt')
y = np.where(dfy.RelapseGroup == 'Relapse', 1, 0)
y[:5]

array([1, 1, 0, 0, 1])

In [5]:
dfyT = pd.read_table('../data/brcaTestPheno.txt')
yT = np.where(dfyT.RelapseGroup == 'Relapse', 1, 0)
yT[:5]

array([1, 0, 1, 0, 0])

### Wilcoxon Rank-Sum Test

I have 22,215 genes. That's *a lot* of genes and not every single one of these genes is going to be predictive of relapse. I turn to the Wilcoxon Rank-Sum test to filter out some of the genes. This is a non-parametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample.

My null hypothesis here is that the NoRelapse patients and the Relapse patients come from the same distribution. I test this hypothesis for all 22,215 genes and only keep genes where the p-value from the test is small enough where I can say that the 2 sets of patients do *not* come from the same population.

In [6]:
def WRST(X,y): #X is feature matrix, y is binary label vector

    num_zeros = sum(y == 0) # Number of NoRelapse patients
    num_ones = len(y) - num_zeros # Number of Relapse patients
    
    #Use Wilcoxon rank-sum test to find most differentially expressed genes
    W = np.array(range(X.shape[1])) #Create W vector whose length = # of genes (aka number of columns)
    
    for i in range(X.shape[1]): #for each gene
        ranks = ss.rankdata(X[:,i],method='average')  # rank patients from least expressed to most expressed
        
        W[i] = sum(ranks[np.nonzero(y)]) #Add together gene i's rank for all patients who relapse
    
    #For binary label, sum up positive labels & negative labels (this equals the mean)
    # When the two samples are sufficiently large, the W statistic is approximately normal N(μ, σ) where
    m = num_ones * (len(y) + 1) / 2. # mu =  n1 * (n + 1) / 2 and 
    s = math.sqrt((num_zeros * num_ones * (len(y) + 1)) / 12.) # var = n1 * n2 * (n + 1) / 12
    
    pval = np.array(range(X.shape[1]), float) # Create empty pval array
    
    #calculate p-values for each gene
    for i in range(len(W)):
        if W[i] <= m:
            pval[i] = 2.*ss.norm.cdf(W[i],m,s)
        else:
            pval[i] = 2.*(1-ss.norm.cdf(W[i],m,s))
    
    #sort p-values
    sort = np.argsort(pval)
    
    return sort

In [7]:
sort = WRST(X, y)

In [8]:
# Create permutated dfs from most differentially expressed to least differentially expressed
dfw = df[df.columns[sort]]
dfTw = dfT[dfT.columns[sort]]
Xw = np.array(dfw)
XTw = np.array(dfTw)

In [9]:
# Rank training set
order = X.argsort(axis=1)
R = X.argsort(axis=1)
dfR = pd.DataFrame(R, index = df.index, columns=df.columns)

# Rank permutated training set
order2 = Xw.argsort(axis=1)
Rw = Xw.argsort(axis=1)
dfRw = pd.DataFrame(Rw, index = dfw.index, columns=dfw.columns)

# Rank testing set
orderT = XT.argsort(axis=1)
RT = XT.argsort(axis=1)
dfRT = pd.DataFrame(RT, index = df.index, columns=df.columns)

# Rank permutated testing set
order2T = XTw.argsort(axis=1)
RTw = XTw.argsort(axis=1)
dfRTw = pd.DataFrame(RTw, index = dfTw.index, columns=dfTw.columns)

## Top Scoring Pairs Classifier

In [10]:
def nCr(n,r):
    f = math.factorial
    return float(f(n) / f(r) / f(n-r))

In [11]:
def TSP(B, z):
    numpairs = nCr(df.shape[1], 2)
    p2 = np.zeros([numpairs, 4]) # pre-allocate space for pij_0 and pij_1
    f = 0 # count

    for i in range(df.shape[1] - 1): # for the first p - 1 genes (every gene except the last one)
        for j in range(i,len(B[1,:]) - 1): # from gene i to 2nd to last gene
            gp = B[:,[i,j+1]] # find gene pairs
            p = np.empty([2, 3], dtype = int) # pre-allocate table to score gene pair
            p[0][2] = len(z) - np.count_nonzero(z) # total profiles with class 0
            p[1][2] = np.count_nonzero(z) # total profiles with class 1
            p[0][0] = 0
            p[0][1] = 0
            p[1][0] = 0
            p[1][1] = 0

            # in each gene pair (i,j)
            for a in range(len(gp)):
                for b in range(gp.shape[1] - 1):
                    if z[a] == 0: # if sample has class 0
                        if gp[a][b] < gp[a][b+1]: # if sample's gene i < gene j
                            p[0][0] += 1 # Add 1 to class 0, Xi < Xj
                        else:
                            p[0][1] += 1 # class 0, rank gene i >= rank gene j, add to table
                    else:
                        if gp[a][b] < gp[a][b+1]:
                            p[1][0] += 1
                        else:
                            p[1][1] += 1

            pij_0 = float(p[0][0]) / p[0][2]
            pij_1 = float(p[1][0]) / p[1][2]

            #del_ij = abs(pij_0 - pij_1) # calculate score for gene pair (i,j)

            # create probability table
            if pij_0 != 0 or pij_1 != 0:
                p2[f][0] = i
                p2[f][1] = j+1
                p2[f][2] = pij_0
                p2[f][3] = pij_1
                f += 1
    p2 = p2[:f,:]

    delta2 = []

    for l in range(len(p2)):
        if abs(p2[l][2] - p2[l][3]) != 0:
            delta2.append(abs(p2[l][2] - p2[l][3]))         
    delta2 = sorted(delta2, reverse = True)

    for k in range(len(p2)):
        if abs(p2[k][2] - p2[k][3]) == delta2[0]:
            g = k
            
    return p2, g

In [12]:
% time p2, g = TSP(Rw[:,:5000], y)

/Users/avinashtamby/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


CPU times: user 4h 38min 2s, sys: 2min 43s, total: 4h 40min 46s
Wall time: 7h 46min 8s


In [20]:
# 100 = 7.26s
# 200 = 34s, 27.4s
# 250 = 1min 3s, 52.9s
# 300 = 1min 22s
# 2500 = 1h 10min 54s
# 3200 = 2h 46min 44s 5,118,400 aka 3200 choose 2
# 5000 = 7h 46min 8s start 8:45am 12,497,500 aka 5000 choose 2

# 22215 = ??? long time - 5 days? ??? 246,742,005 aka 22215 choose 2

# np.savetxt('twopoint5k.csv', p2, delimiter=',') 312.4 MB
# np.savetxt('threepoint2k.csv', p2, delimiter=',') 511.8 MB
# np.savetxt('fivek.csv', p2, delimiter=',') 1.25 GB

# According to my calculations, the file with all 22,215 genes will be about 25 GB

In [13]:
p2[g]

array([  3.20000000e+01,   2.79100000e+03,   5.85526316e-01,
         1.83333333e-01])

In [14]:
len(p2)

12497500

In [15]:
def acc(B, z, p2, g):
    import warnings
    warnings.filterwarnings("ignore")

    g1 = p2[g][0]
    g2 = p2[g][1]
    ypred = np.zeros(len(z), dtype = int)
    count = 0
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for h in range(len(B)):
        if p2[g][2] > p2[g][3]:
            if B[h,g1] < B[h,g2]:
                ypred[h] = 0
            else:
                ypred[h] = 1
        else:
            if B[h,g1] > B[h,g2]:
                ypred[h] = 0
            else:
                ypred[h] = 1

        if z[h] == ypred[h]:
            count += 1

        # Positive = Relapse = 1
        if z[h] == 1 and ypred[h] == 1:
            TP += 1
        elif z[h] == 1 and ypred[h] == 0:
            FN += 1
        elif z[h] == 0 and ypred[h] == 1:
            FP += 1
        else:
            TN += 1

    conmat = np.array([[TP, FN], [FP, TN]])

    TPR = float(TP) / (TP + FN) # Sensitivity: % of Relapse individuals correctly classified as Relapse
    FPR = float(FP) / (TN + FP) # 1 - Specificity: % of NoRelapse individuals incorrectly classified as Relapse

    # TNR = TN / (TN + FP) # Specificity: % of NoRelapse individuals correctly classified as NoRelapse
    # FNR = FN / (TP + FN) # 1 - TPR: % of Relapse individuals incorrectly classified as NoRelapse

    # PPR = TP / (TP + FP) # Pos Predictive Value: % of correctly classified NoRelapse predictions
    # NPR = TN / (TN + FN) # Neg Predictive Value: % of correctly classified Relapse predictions

    # FDR = FP / (FP + TP) # % of incorrectly classified NoRelapse predictions
    #print('TPR = ', TPR)
    #print('FPR = ', FPR)
    #print('score = ',(count/len(X)))
    accu = count/float(len(X))
    
    return ypred, accu, conmat

In [22]:
tic = time()
accur = []
for i in range(len(p2)):
    accur.append(acc(RTw[:,:5000], yT, p2, i)[1])
toc = time()
toc - tic

KeyboardInterrupt: 

DNA → RNA → Protein: The Central Dogma
Proteins created from mutated DNA prevent cells from functioning normally
Expression levels collected by DNA microarray
Collection of microscopic DNA spots attached to a solid surface
Each DNA spot contains a small amount of a specific DNA sequence (or gene) known as a probe
Probes can then link with the target gene that you want to analyze
Probe-target links can then be detected and quantified to measure expression level of a gene
Different labs can yield different expression levels for the same gene
The data that I analyze come from 3 different studies
Data is riddled with inconsistent measurements


In [ ]:
max(accur)

In [ ]:
# % time p3 = np.loadtxt('twopoint5k.csv', delimiter=',')

In [16]:
ypred, accu, conmat = acc(RTw[:,:5000], yT, p2, g)

In [ ]:
ypred, accu, conmat = acc(RTw[:,:5000], yT, p2, accur.index(max(accur)))

In [17]:
confusion = pd.DataFrame(conmat, index=['Relapse', 'NoRelapse'],
                         columns=['predicted_Relapse','predicted_NoRelapse'])
confusion

,predicted_Relapse,predicted_NoRelapse
Relapse,45,30
NoRelapse,68,69


In [18]:
print 'Null Accuracy:', 137./212
print 'True Accuracy:', accu

Null Accuracy: 0.646226415094
True Accuracy: 0.537735849057


In [19]:
print 'Sensitivity/Recall/TPR (Correct Relapse Predictions / Total Relapse):', float(conmat[0][0]) / conmat.sum(axis=1)[0]
print ''
print 'Specificity/TNR (Correct NoRelapse Predictions / Total NoRelapse):', float(conmat[1][1]) / conmat.sum(axis=1)[1]
print ''
print 'Precision (Correct Relapse Predictions / Total Relapse Predictions):', float(conmat[0][0]) / conmat.sum(axis=0)[0]
print ''
print 'FPR (Incorrect Relapse Predictions / Total NoRelapse):', float(conmat[1][0]) / conmat.sum(axis=1)[1]

Sensitivity/Recall/TPR (Correct Relapse Predictions / Total Relapse): 0.6

Specificity/TNR (Correct NoRelapse Predictions / Total NoRelapse): 0.503649635036

Precision (Correct Relapse Predictions / Total Relapse Predictions): 0.398230088496

FPR (Incorrect Relapse Predictions / Total NoRelapse): 0.496350364964


In [ ]:
ypred

## Random Forest Classifier

In [ ]:
cv_params = {'max_depth': [3,5,7]}

ind_params = {'n_estimators': 100}

optimized_rf = GridSearchCV(RandomForestClassifier(**ind_params), cv_params, cv = 5, n_jobs = -1)

In [ ]:
% time optimized_rf.fit(Xw, y)

In [ ]:
rf_pp = optimized_rf.predict_proba(XTw)
print(optimized_rf.score(XTw, yT))

In [ ]:
conmat = np.array(confusion_matrix(yT, optimized_rf.predict(XTw[:,:10]), labels=[1,0]))

confusion = pd.DataFrame(conmat, index=['Relapse', 'NoRelapse'],
                         columns=['predicted_Relapse','predicted_NoRelapse'])

confusion

In [ ]:
fpr, tpr, thresholds = roc_curve(yT, rf_pp[:,1])
roc_auc = auc(fpr, tpr)
%matplotlib inline
plt.plot(fpr, tpr, label='AUC = %0.3f' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate or (1 - Specificity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic for Training Data')
plt.legend(loc="lower right")
plt.show()

print roc_auc

## XGBoost Classifier

In [ ]:
xgb.XGBClassifier()

In [ ]:
cv_params = {'max_depth': [3,5,7,9,11,13,15], 'learning_rate': [0.01, 0.05, 0.1, 0.5],  'min_child_weight': [1,2,3,4,5,6,7],
             'subsample': [0.2, 0.4, 0.6, 0.8]}

ind_params = {'n_estimators': 5000, 'silent': False, 'nthread': -1,  'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}

optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), cv_params, scoring = 'roc_auc', cv = 5, n_jobs = -1)

In [ ]:
optimized_GBM.fit(Xw, y)

In [ ]:
test_pp = optimized_GBM.predict_proba(XTw)
print(optimized_GBM.score(XTw, yT))

In [ ]:
fpr, tpr, thresholds = roc_curve(yT, test_pp[:,1])
roc_auc = auc(fpr, tpr)
%matplotlib inline
plt.plot(fpr, tpr, label='AUC = %0.3f' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate or (1 - Specificity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic for Training Data')
plt.legend(loc="lower right")
plt.show()

print roc_auc

In [ ]:
answer = range(22215)
r = [i ** 2 * .00056 for i in answer]

% matplotlib inline

for i in range(answer[115], answer[-1], 1000):
    if r[i] < 60:
        print(i, 'genes will take ', r[i], 'seconds')
    elif r[i] >= 60 and answer[i] < 3600:
        mins = r[i]/60
        sec = (mins - int(mins)) * 60
        print(i, 'genes will take', int(mins), 'minutes and', sec, 'seconds')
    elif r[i] >= 3600 and r[i] < 86400:
        hr = r[i]/3600
        mins = (hr - int(hr)) * 60
        print(i, 'genes will take', int(hr), 'hours', int(mins), 'minutes')
    else:
        days = r[i]/86400
        hr = (days - int(r[i]/86400))*24
        print(i, 'genes will take', int(days), 'days', int(hr), 'hours')

plt.scatter(answer, r)
plt.xlabel('Number of Genes')
plt.ylabel('Time Elapsed')
plt.show()